In [8]:
# !pip install -r requirements.txt

  Using cached geopy-2.2.0-py3-none-any.whl (118 kB)


ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [1]:
import spacy
import pandas as pd
import functions
from alive_progress import alive_bar
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, TextClassificationPipeline, pipeline)

# Reading and cleaning all texts

In [2]:
#geolocated_texts = pd.read_csv('./../data/texts.csv')
no_geolocated_texts = pd.read_csv('./../data/texts/all_texts.csv')

In [ ]:
#full_texts = pd.concat([geolocated_texts, no_geolocated_texts])
full_texts = no_geolocated_texts
full_texts['country'] = 'no_geolocated'

In [3]:
no_geolocated_texts

,Id,text,date,country
0,429478735541460992,RT @Veneco_Babes: @marycarmenfh @trinysex @Div...,2014-01-01,no_geolocated
1,429476144170422272,RT @Veneco_Babes: @marycarmenfh @trinysex @Div...,2014-01-01,no_geolocated
2,429475247448793088,@aluxtuitea @rogeliosamour @JJRENDON @JNatanVa...,2014-01-01,no_geolocated
3,429443798234824704,@Mayomendozaf @callollanos @JanioCuadro el que...,2014-01-01,no_geolocated
4,429438621037625344,A ustedes les gusta que les digan Venecos/as?,2014-01-01,no_geolocated
...,...,...,...,...
7406214,1531863429325144064,"RT @kamizvlog: Les llaman ""Venecos"" a los Vene...",2022-06-01,no_geolocated
7406215,1531863330910085120,RT @PepBohrquez: @Kattycita10 Lo peor es la id...,2022-06-01,no_geolocated
7406216,1531863314078244866,"RT @ElPitazoTV: #Migración | Anaís Bolívar, ps...",2022-06-01,no_geolocated
7406217,1531863300874678273,"RT @kamizvlog: Les llaman ""Venecos"" a los Vene...",2022-06-01,no_geolocated


In [4]:
full_texts['text'] = full_texts['text'].apply(functions.tweet_text_clean)

# Model 0 geolocation

In [5]:
nlp = spacy.load('./model0_geolocation/model-last')

In [10]:
colombian_tweets = []
with alive_bar(len(full_texts), force_tty = True) as bar: 
    for index,row in full_texts.iterrows():
        if row['country'] in ['no_geolocated', 'no_location', 'no_country']:
            row['country'] = functions.predict_cat(row['text'], nlp)
        if row['country'] == 'Colombia':
            colombian_tweets.append({'Id':row['Id'],'date':row['date'], 'text':row['text']})  # Add the tweet to the list of tweets from Colombia
        bar()

|████████████████████████████████████████| 7406219/7406219 [100%] in 1:42:01.9 (1209.80/s)                               ▂▂▄ 9204/7406219 [0%] in 9s (1073.4/s, eta: 1:54:58)  █▆▄ 70094/7406219 [1%] in 1:00 (1162.9/s, eta: 1:45:07)  ▆▄▂ 86752/7406219 [1%] in 1:15 (1158.9/s, eta: 1:45:17)  ▆█▆ 95277/7406219 [1%] in 1:22 (1163.5/s, eta: 1:44:44)  ▄▂▂ 106907/7406219 [1%] in 1:32 (1161.9/s, eta: 1:44:41)  ▄▂▂ 178011/7406219 [2%] in 2:30 (1187.8/s, eta: 1:41:25)  ▂▂▄ 204046/7406219 [3%] in 2:50 (1198.0/s, eta: 1:40:12)  ▃▅▇ 205928/7406219 [3%] in 2:52 (1198.8/s, eta: 1:40:06)  ▅▇▇ 241859/7406219 [3%] in 3:20 (1208.8/s, eta: 1:38:47)  ▇▇▅ 260939/7406219 [4%] in 3:35 (1212.1/s, eta: 1:38:15)  ▃▁▃ 292332/7406219 [4%] in 4:00 (1218.7/s, eta: 1:37:17)  ▇▅▃ 348575/7406219 [5%] in 4:44 (1228.9/s, eta: 1:35:43)  ▂▂▄ 372294/7406219 [5%] in 5:02 (1231.6/s, eta: 1:35:11)  ▃▁▃ 396460/7406219 [5%] in 5:21 (1234.1/s, eta: 1:34:40)  ▆▄▂ 443072/7406219 [6%] in 5:59 (1235.7/s, eta: 1:33:55)  ▄▂▂ 477417/74062

In [12]:
pd.DataFrame.from_dict(colombian_tweets).to_csv('./../data/texts/colombian_tweets.csv', index=False)

# Model 1 prediction - valid tweets

In [14]:
nlp2 = spacy.load('./model1_valid/es_core_news_sm/model-last')

In [15]:
colombian_valid_tweets = []
with alive_bar(len(colombian_tweets), force_tty = True) as bar: 
    for row in colombian_tweets:
        valid = functions.predict_cat(row['text'], nlp2)
        if valid == '1':
            colombian_valid_tweets.append({'Id':row['Id'],'date':row['date'], 'text':row['text']})  # Add the tweet to the list of tweets from Colombia
        bar()

|████████████████████████████████████████| 2006528/2006528 [100%] in 22:03.5 (1516.09/s)                                 ▄▂▂ 35395/2006528 [2%] in 30s (1189.6/s, eta: 27:37)  ▇▅▃ 45114/2006528 [2%] in 37s (1223.3/s, eta: 26:44)  ▂▄▆ 46570/2006528 [2%] in 38s (1229.9/s, eta: 26:34)  ▆▄▂ 48047/2006528 [2%] in 39s (1234.9/s, eta: 26:26)  ▂▂▄ 48331/2006528 [2%] in 39s (1236.2/s, eta: 26:24)  ▄▂▂ 67253/2006528 [3%] in 52s (1290.1/s, eta: 25:03)  ▅▇▇ 71413/2006528 [4%] in 55s (1296.5/s, eta: 24:53)  ▄▂▂ 82303/2006528 [4%] in 1:03 (1313.8/s, eta: 24:25)  ▃▅▇ 89725/2006528 [4%] in 1:07 (1329.8/s, eta: 24:02)  ▄▆█ 145452/2006528 [7%] in 1:44 (1395.4/s, eta: 22:14)  ▃▁▃ 183823/2006528 [9%] in 2:10 (1418.6/s, eta: 21:25)  ▂▂▄ 185909/2006528 [9%] in 2:11 (1419.8/s, eta: 21:22)  ▇▅▃ 272267/2006528 [14%] in 3:08 (1449.0/s, eta: 19:57)  ▅▇▇ 278845/2006528 [14%] in 3:12 (1450.0/s, eta: 19:52)  ▁▃▅ 291018/2006528 [15%] in 3:20 (1455.2/s, eta: 19:39)  ▇▅▃ 457950/2006528 [23%] in 5:09 (1481.0/s, eta: 17:

In [16]:
pd.DataFrame.from_dict(colombian_valid_tweets).to_csv('./../data/texts/colombian_valid_tweets.csv', index=False)

# Model 2 prediction - referred_to

### Prediction with spacy model

In [19]:
nlp3 = spacy.load('./model2_subject/es_core_news_sm/model-last')

In [20]:
colombian_valid_tweets_predictions = []
with alive_bar(len(colombian_valid_tweets), force_tty = True) as bar: 
    for row in colombian_valid_tweets:
        referred_to = functions.predict_cat(row['text'], nlp3)
        colombian_valid_tweets_predictions.append({'Id':row['Id'],'date':row['date'], 'text':row['text'], 'referred_to':referred_to})  # Add the tweet to the list of tweets from Colombia
        bar()

|████████████████████████████████████████| 1664903/1664903 [100%] in 37:20.0 (743.26/s)                                 s (645.5/s, eta: 42:50)  ▄▂▂ 8435/1664903 [1%] in 12s (678.3/s, eta: 40:42)  ▆▄▂ 12611/1664903 [1%] in 18s (691.1/s, eta: 39:51)  ▂▄▆ 55248/1664903 [3%] in 1:12 (770.0/s, eta: 34:51)  ▄▆█ 129941/1664903 [8%] in 2:43 (796.5/s, eta: 32:07)  ▆█▆ 136560/1664903 [8%] in 2:51 (798.0/s, eta: 31:55)  ▃▁▃ 173504/1664903 [10%] in 3:36 (802.8/s, eta: 30:58)  ▃▅▇ 175753/1664903 [11%] in 3:39 (802.8/s, eta: 30:55)  ▆▄▂ 184001/1664903 [11%] in 3:49 (803.1/s, eta: 30:44)  ▁▃▅ 186762/1664903 [11%] in 3:53 (802.9/s, eta: 30:41)  █▆▄ 242637/1664903 [15%] in 5:02 (803.6/s, eta: 29:30)  ▇▅▃ 266094/1664903 [16%] in 5:30 (807.5/s, eta: 28:52)  ▅▃▁ 266718/1664903 [16%] in 5:30 (807.5/s, eta: 28:52)  ▃▁▃ 275067/1664903 [17%] in 5:40 (808.5/s, eta: 28:39)  ▇▅▃ 306255/1664903 [18%] in 6:18 (809.9/s, eta: 27:58)  ▆█▆ 361218/1664903 [22%] in 7:26 (810.6/s, eta: 26:48)  ▃▅▇ 369477/1664903 [22%] i

In [21]:
pd.DataFrame.from_dict(colombian_valid_tweets_predictions)['referred_to'].value_counts()

migrants       700420
geopolitics    387442
government     319299
migration      257742
Name: referred_to, dtype: int64

In [22]:
pd.DataFrame.from_dict(colombian_valid_tweets_predictions).to_csv('./../data/texts/colombian_valid_tweets_subjectpreds_spacy.csv', index=False)

### Prediction with bert-base-spanish-wwm-cased model

In [16]:
colombian_valid_tweets = pd.read_csv('./../data/texts/colombian_valid_tweets.csv')

In [17]:
colombian_valid_tweets

,Id,date,text
0,429475247448793088,2014-01-01,aluxtuitea rogeliosamour JJRENDON JNatanVaquiz...
1,429443798234824704,2014-01-01,Mayomendozaf callollanos JanioCuadro el que no...
2,429435589205585920,2014-01-01,Recuerdos de mis bellos sobrino Venecos
3,429401602966171649,2014-01-01,"SienMisManosEstuviera Cerraría la frontera, de..."
4,429394529024622592,2014-01-01,ACUATheragram ACUATheragram Es URIBE O SINO N...
...,...,...,...
1664898,1531865786754682880,2022-06-01,miremejias Y se olvidan del sector de La Maris...
1664899,1531865341399375875,2022-06-01,Veneco nivel
1664900,1531863973582327808,2022-06-01,"Si no nos matan los venecos, nos mata el escap..."
1664901,1531863579875581953,2022-06-01,Me encanta ver venecos de trending y cómo nos ...


In [19]:
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = AutoModelForSequenceClassification.from_pretrained("./model2_subject/bert_wwm/")

In [27]:
pipe_nlp2 = pipeline("text-classification", model=model, tokenizer = tokenizer)

In [ ]:
colombian_valid_tweets_subject_predictions = []
with alive_bar(len(colombian_valid_tweets), force_tty = True) as bar: 
    for index,row in colombian_valid_tweets.iterrows():
        referred_to = pipe_nlp2(str(row['text']))[0]['label']
        colombian_valid_tweets_subject_predictions.append({'Id':row['Id'],'date':row['date'], 'text':row['text'], 'referred_to':referred_to})  # Add the tweet to the list of tweets from Colombia
        bar()

In [ ]:
pd.DataFrame.from_dict(colombian_valid_tweets_subject_predictions).to_csv('./../data/texts/colombian_valid_tweets_subject_preds_wwm.csv', index=False)

# Model 3 prediction - tone

### Prediction with bert-base-spanish-wwm-cased model

In [53]:
colombian_valid_tweets_subject_predictions = pd.read_csv('./../data/texts/colombian_valid_tweets_subject_preds_wwm.csv') #can be replaced by colombian_valid_tweets

In [55]:
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = AutoModelForSequenceClassification.from_pretrained("./model3_tone/bert_wwm/")

In [56]:
pipe_nlp3 = pipeline("text-classification", model=model, tokenizer = tokenizer)

In [59]:
colombian_valid_tweets_tone_predictions = []
with alive_bar(len(colombian_valid_tweets_subject_predictions), force_tty = True) as bar: 
    for index,row in colombian_valid_tweets_subject_predictions.iterrows():
        tone = pipe_nlp3(str(row['text']))[0]['label']
        colombian_valid_tweets_tone_predictions.append({'Id':row['Id'],'date':row['date'], 'referred_to':row['referred_to'],
            'text':row['text'], 'tone':tone})  # Add the tweet to the list of tweets from Colombia
        bar()

|████████████████████████████████████████| 1664903/1664903 [100%] in 18:40:22.9 (24.77/s)                               %] in 44:04 (22.9/s, eta: 19:29:23)  ▇▇▅ 82515/1664903 [5%] in 58:57 (23.3/s, eta: 18:50:38)  ▇▅▃ 88030/1664903 [5%] in 1:02:38 (23.4/s, eta: 18:41:51)  ▁▃▅ 89268/1664903 [5%] in 1:03:27 (23.4/s, eta: 18:39:54)  ▆▄▂ 92213/1664903 [6%] in 1:05:25 (23.5/s, eta: 18:35:34)  ▆▄▂ 96080/1664903 [6%] in 1:07:55 (23.6/s, eta: 18:28:56)  ▂▂▄ 96253/1664903 [6%] in 1:08:01 (23.6/s, eta: 18:28:34)  ▇▅▃ 96526/1664903 [6%] in 1:08:12 (23.6/s, eta: 18:28:08)  ▆█▆ 97278/1664903 [6%] in 1:08:40 (23.6/s, eta: 18:26:28)  ▇▅▃ 98483/1664903 [6%] in 1:09:23 (23.7/s, eta: 18:23:31)  ▃▁▃ 99509/1664903 [6%] in 1:10:00 (23.7/s, eta: 18:21:16)  ▁▃▅ 100559/1664903 [6%] in 1:10:45 (23.7/s, eta: 18:20:31)  ▄▆█ 100977/1664903 [6%] in 1:11:01 (23.7/s, eta: 18:20:01)  ▅▃▁ 101283/1664903 [6%] in 1:11:12 (23.7/s, eta: 18:19:14)  ▅▇▇ 101369/1664903 [6%] in 1:11:15 (23.7/s, eta: 18:18:56)  ▆█▆ 126133/1664

In [60]:
pd.DataFrame.from_dict(colombian_valid_tweets_tone_predictions).to_csv('./../data/texts/colombian_valid_tweets_tone_preds_wwm.csv', index=False)

# Model 4 prediction - negativeness

### Prediction with bert-base-multilingual-uncased-sentiment model

In [2]:
colombian_valid_tweets_tone_predictions = pd.read_csv('./../data/texts/colombian_valid_tweets_tone_preds_wwm.csv') #can be replaced by colombian_valid_tweets

In [3]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("./model4_negativeness/bert_multi/")

In [4]:
pipe_nlp4 = pipeline("text-classification", model=model, tokenizer = tokenizer)

In [5]:
colombian_valid_tweets_negativeness_predictions = []
for index,row in colombian_valid_tweets_tone_predictions.iterrows():
    if row['tone'] == 'negative':
        negativeness = pipe_nlp4(str(row['text']))[0]['label']
    else:
        negativeness = 'NA'
    colombian_valid_tweets_negativeness_predictions.append({'Id':row['Id'],'date':row['date'], 'referred_to':row['referred_to'],
            'text':row['text'], 'tone':row['tone'], 'negativeness':negativeness})  # Add the tweet to the list of tweets from Colombia

In [6]:
pd.DataFrame.from_dict(colombian_valid_tweets_negativeness_predictions).to_csv('./../data/texts/colombian_valid_tweets_negativeness_preds_multi.csv', index=False)